In [1]:
from os import path
import pandas as pd
import numpy as np
import json
from openai import OpenAI
from tqdm import tqdm
from dotenv import load_dotenv
from sklearn.cluster import KMeans
from sklearn.metrics import pairwise_distances_argmin_min
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

from mint.config import DATA_DIR, RESULT_DIR

c:\Users\Admin\Python\ABSA_Prompting\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
load_dotenv

<function dotenv.main.load_dotenv(dotenv_path: Union[str, ForwardRef('os.PathLike[str]'), NoneType] = None, stream: Optional[IO[str]] = None, verbose: bool = False, override: bool = False, interpolate: bool = True, encoding: Optional[str] = 'utf-8') -> bool>

In [3]:
# CONFIG
ViABSA_BP_dir = path.join(DATA_DIR, 'ViABSA_BP')
test_file = path.join(ViABSA_BP_dir, 'data_test.csv')
test_df = pd.read_csv(test_file)

In [4]:
def transform_aspect_sentiment(df, start=0, end=None):
    aspects = [
        "stayingpower",
        "texture",
        "smell",
        "price",
        "others",
        "colour",
        "shipping",
        "packing"
    ]

    if end is None:
        end = len(df)

    result = []

    for idx, row in df.iloc[start:end].iterrows():
        entry = {
            "id": str(idx),
            "text": row['data'],
            "sentiments": []
        }

        for aspect in aspects:
            sentiment = row[f"{aspect}_label"]
            if sentiment == 1: 
                aspect_sentiment_value = row[aspect]
                if aspect_sentiment_value != 'none':
                    entry["sentiments"].append({
                        "aspect": aspect,
                        "sentiment": aspect_sentiment_value
                    })
                else:
                    entry["sentiments"].append({
                        "aspect": aspect,
                        "sentiment": "unknown"
                    })

        result.append(entry)

    return result

In [5]:
# SETUP DATA
aspects = ['stayingpower', 'texture', 'smell', 'price', 'others', 'colour', 'shipping', 'packing']
test_df[aspects] = test_df[aspects].fillna('none')

for aspect in aspects:
    test_df[aspect + '_label'] = (test_df[aspect] != 'none').astype(int)

test_json = transform_aspect_sentiment(test_df, 0, 100)
test_json[:5]

[{'id': '0',
  'text': 'Hàng đóng gói đẹp và chắc chắn, nhìn rất dễ thương và ưng bụng ạ!',
  'sentiments': [{'aspect': 'packing', 'sentiment': 'positive'}]},
 {'id': '1',
  'text': 'Cảm giác son bên trong rất là ít luôn í, đóng gói cẩn thận, dịch nhưng mà giao hàng khá nhanh',
  'sentiments': [{'aspect': 'shipping', 'sentiment': 'positive'},
   {'aspect': 'packing', 'sentiment': 'positive'}]},
 {'id': '2',
  'text': 'Son siêu đẹp luôn ý, mà y hình mà chụp có thể không giống lắm nhưng nhìn ngoài thì giống nha. Chất son mềm mướt nói chung là rất thíchhhh',
  'sentiments': [{'aspect': 'texture', 'sentiment': 'positive'}]},
 {'id': '3',
  'text': 'Siu đẹp luôn \r\nShop đóng gói cẩn thận lắm luôn \r\nLại thêm cả quà nữa\r\nNói chung là thích lắm',
  'sentiments': [{'aspect': 'packing', 'sentiment': 'positive'}]},
 {'id': '4',
  'text': 'Aúhihđcyihb gfxxth jj bhgfzđE G GHVHBTCEETXUBBIYCZRZRTCVUUBYRZXGVBIINVUTXZRCTIBONONBUXTZRTVKNNOUVTXEecyuvknibtcrztxuvbubibijvcytdgunonobiyvdtrdfyyghuojpmib

**Ý tưởng chọn example cho few-shot**
1.   Chọn ra top-k với ví dụ gần nhất với input hiện tại
2.   Tạo prompt khác nhau cho từng input

Áp dụng retrieval-based dynamic few-shot prompting 

In [49]:
def get_top_k_examples(candidate_examples, review_to_predict, k=3, model_name='all-MiniLM-L6-v2'):
    # Lọc lại để không trùng text với input
    filtered_candidates = [ex for ex in candidate_examples if ex["text"].strip() != review_to_predict.strip()]

    model = SentenceTransformer(model_name)

    example_texts = [ex['text'] for ex in filtered_candidates]
    all_texts = example_texts + [review_to_predict]
    embeddings = model.encode(all_texts)

    # lấy vector cuối
    review_emb = embeddings[-1].reshape(1, -1)
    # lấy tất cả trừ vector cuối
    example_embs = embeddings[:-1]
    
    # tính cosine similarity
    sims = cosine_similarity(review_emb, example_embs)[0]
    top_k_idx = np.argsort(sims)[-k:][::-1]

    return [candidate_examples[i] for i in top_k_idx]


def build_dynamic_prompt(review_to_predict, top_examples):
    prompt = 'Extract aspects and sentiments from the following review:.\n\n'
    prompt += f'Data: {review_to_predict}\n'
    prompt += 'Examples:\n'
    for ex in top_examples:
        if ex["text"].strip() != review_to_predict.strip():  # Bỏ ví dụ trùng
            prompt += f'Review: "{ex["text"]}"\nOutput: {json.dumps(ex["sentiments"], ensure_ascii=False)}\n\n'
    prompt += f'Review: "{review_to_predict}"\nOutput:'
    return prompt

In [34]:
top_examples = get_top_k_examples(test_json, "Hàng đóng gói đẹp và chắc chắn, nhìn rất dễ thương và ưng bụng ạ!", k=5, model_name='all-MiniLM-L6-v2')
top_examples[:5]

[{'id': '27',
  'text': 'Đóng gói ok, 02 mùi khá thơm, son ko đc đầy cho lắm, vs giá đó thì ko mong nhiều',
  'sentiments': [{'aspect': 'smell', 'sentiment': 'positive'},
   {'aspect': 'price', 'sentiment': 'positive'},
   {'aspect': 'packing', 'sentiment': 'positive'}]},
 {'id': '81',
  'text': 'Hehee ảnh của chì kẻ mắt nma son xịn ghê lun mng màu xinh á 🐶🐶🐶🙈🙈🙈🙈🙈',
  'sentiments': [{'aspect': 'colour', 'sentiment': 'positive'}]},
 {'id': '42',
  'text': 'Hình ảnh mang tính chất minh hoạ 🌸🌸💕🌸💕🌸💕🌸💕🌸💕😳💕🌸💕🌸🌸💕🌸🌸💕🌸🌸🌸🌸🌸🌸',
  'sentiments': [{'aspect': 'others', 'sentiment': 'neutral'}]},
 {'id': '72',
  'text': 'rất ok\n\n\nuuhshduddhd',
  'sentiments': [{'aspect': 'others', 'sentiment': 'neutral'}]},
 {'id': '38',
  'text': 'Giao hàng nhanh lắm ạ. Chỉ mong các shop nước ngoài cũng nhanh như shop này thôi. Ưng thật sự',
  'sentiments': [{'aspect': 'shipping', 'sentiment': 'positive'}]}]

In [52]:
prompt = build_dynamic_prompt("Hàng đóng gói đẹp và chắc chắn, nhìn rất dễ thương và ưng bụng ạ!", top_examples)
print(prompt)

Extract aspects and sentiments from the following review:.

Data: Hàng đóng gói đẹp và chắc chắn, nhìn rất dễ thương và ưng bụng ạ!
Examples:
Review: "Đóng gói ok, 02 mùi khá thơm, son ko đc đầy cho lắm, vs giá đó thì ko mong nhiều"
Output: [{"aspect": "smell", "sentiment": "positive"}, {"aspect": "price", "sentiment": "positive"}, {"aspect": "packing", "sentiment": "positive"}]

Review: "Hehee ảnh của chì kẻ mắt nma son xịn ghê lun mng màu xinh á 🐶🐶🐶🙈🙈🙈🙈🙈"
Output: [{"aspect": "colour", "sentiment": "positive"}]

Review: "Hình ảnh mang tính chất minh hoạ 🌸🌸💕🌸💕🌸💕🌸💕🌸💕😳💕🌸💕🌸🌸💕🌸🌸💕🌸🌸🌸🌸🌸🌸"
Output: [{"aspect": "others", "sentiment": "neutral"}]

Review: "rất ok


uuhshduddhd"
Output: [{"aspect": "others", "sentiment": "neutral"}]

Review: "Giao hàng nhanh lắm ạ. Chỉ mong các shop nước ngoài cũng nhanh như shop này thôi. Ưng thật sự"
Output: [{"aspect": "shipping", "sentiment": "positive"}]

Review: "Hàng đóng gói đẹp và chắc chắn, nhìn rất dễ thương và ưng bụng ạ!"
Output:


In [19]:
gpt_functions = [
    {
        "name": "extract_aspect_sentiment",
        "description": "Extract aspects and sentiments from text",
        "parameters": {
            "type": "object",
            "properties": {
                "results": {
                    "type": "array",
                    "items": {
                        "type": "object",
                        "properties": {
                            "aspect": {
                                "type": "string",
                                "enum": aspects
                            },
                            "sentiment": {
                                "type": "string",
                                "enum": ["positive", "negative", "neutral"]
                            }
                        },
                        "required": ["aspect", "sentiment"]
                    }
                }
            },
            "required": ["results"]
        }
    }
]


In [20]:
def evaluate_aspect_sentiment(ground_truth, predictions):
    true_aspects = []
    pred_aspects = []

    true_aspect_sentiments = []
    pred_aspect_sentiments = []

    for gt_entry, pred_entry in zip(ground_truth, predictions):
        gt_sents = gt_entry['sentiments']
        gt_aspect_set = set()
        gt_aspect_sentiment_set = set()

        for item in gt_sents:
            gt_aspect_set.add(item['aspect'])
            gt_aspect_sentiment_set.add((item['aspect'], item['sentiment']))

        true_aspects.append(gt_aspect_set)
        true_aspect_sentiments.append(gt_aspect_sentiment_set)

        pred_sents = pred_entry['results']
        pred_aspect_set = set()
        pred_aspect_sentiment_set = set()

        for item in pred_sents:
            pred_aspect_set.add(item['aspect'])
            pred_aspect_sentiment_set.add((item['aspect'], item['sentiment']))

        pred_aspects.append(pred_aspect_set)
        pred_aspect_sentiments.append(pred_aspect_sentiment_set)

    all_true_aspects = set.union(*true_aspects) if true_aspects else set()
    all_pred_aspects = set.union(*pred_aspects) if pred_aspects else set()

    tp_aspect = sum(len(gt & pred) for gt, pred in zip(true_aspects, pred_aspects))
    fp_aspect = sum(len(pred - gt) for gt, pred in zip(true_aspects, pred_aspects))
    fn_aspect = sum(len(gt - pred) for gt, pred in zip(true_aspects, pred_aspects))

    precision_aspect = tp_aspect / (tp_aspect + fp_aspect + 1e-8)
    recall_aspect = tp_aspect / (tp_aspect + fn_aspect + 1e-8)
    f1_aspect = 2 * precision_aspect * recall_aspect / (precision_aspect + recall_aspect + 1e-8)

    tp_sentiment = sum(len(gt & pred) for gt, pred in zip(true_aspect_sentiments, pred_aspect_sentiments))
    fp_sentiment = sum(len(pred - gt) for gt, pred in zip(true_aspect_sentiments, pred_aspect_sentiments))
    fn_sentiment = sum(len(gt - pred) for gt, pred in zip(true_aspect_sentiments, pred_aspect_sentiments))

    precision_sentiment = tp_sentiment / (tp_sentiment + fp_sentiment + 1e-8)
    recall_sentiment = tp_sentiment / (tp_sentiment + fn_sentiment + 1e-8)
    f1_sentiment = 2 * precision_sentiment * recall_sentiment / (precision_sentiment + recall_sentiment + 1e-8)

    return {
        "Aspect Detection F1": f1_aspect,
        "Sentiment Classification F1": f1_sentiment
    }


In [53]:
client = OpenAI()
predictions = []

for data in tqdm(test_json):
    review = data['text']

    top_examples = get_top_k_examples(test_json, review, k=5, model_name='all-MiniLM-L6-v2')
    prompt = build_dynamic_prompt(review, top_examples)

    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": "You are an AI assistant that extracts aspects and their sentiments from text."},
            {"role": "user", "content": prompt}
        ],
        functions=gpt_functions,
        function_call={"name": "extract_aspect_sentiment"},
        temperature=0
    )

    # Parse kết quả
    output = response.choices[0].message.function_call.arguments
    parsed_output = json.loads(output)
    predictions.append(parsed_output)

100%|██████████| 100/100 [08:18<00:00,  4.98s/it]


In [54]:
scores = evaluate_aspect_sentiment(test_json, predictions)
print(scores)

{'Aspect Detection F1': 0.8883495095464936, 'Sentiment Classification F1': 0.7632850191475881}


In [55]:
result_file = path.join(RESULT_DIR, 'ViABSA_BP_Few-shot-Similarity.json')
with open(result_file, 'w') as f:
    json.dump(predictions, f, indent=4, ensure_ascii=False)